In [1]:
import requests
import pandas as pd

In [2]:
indicators_to_keep = [
    'NY.GDP.MKTP.CD',
    'NY.GDP.PCAP.CD',
    'FP.CPI.TOTL.ZG',
    'SL.UEM.TOTL.ZS',
    'SP.POP.TOTL',
    'NE.EXP.GNFS.ZS'
]
data = []
for indicator in indicators_to_keep:
    url = f"https://api.worldbank.org/v2/country/all/indicator/{indicator}"
    params = {
        "format": "json",
        "per_page": 20000
    }
    response = requests.get(url, params)
    data = data + response.json()[1]
    if len(json_data) > 1 and json_data[1] is not None:
        data.extend(json_data[1])

df = pd.json_normalize(data)

(103740, 10)

In [4]:
df.head()

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
0,AFE,2024,1.242694e+12,,,0,NY.GDP.MKTP.CD,GDP (current US$),ZH,Africa Eastern and Southern
1,AFE,2023,1.179359e+12,,,0,NY.GDP.MKTP.CD,GDP (current US$),ZH,Africa Eastern and Southern
2,AFE,2022,1.228968e+12,,,0,NY.GDP.MKTP.CD,GDP (current US$),ZH,Africa Eastern and Southern
3,AFE,2021,1.114145e+12,,,0,NY.GDP.MKTP.CD,GDP (current US$),ZH,Africa Eastern and Southern
4,AFE,2020,9.386076e+11,,,0,NY.GDP.MKTP.CD,GDP (current US$),ZH,Africa Eastern and Southern


In [7]:
df = df[['country.value', 'countryiso3code', 'indicator.value', 'indicator.id', 'date', 'value']]
df.columns = ["country_name", "country_code", "indicator_name", "indicator_code", "year", "value"]
df.head()

,country_name,country_code,indicator_name,indicator_code,year,value
0,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2024,1.242694e+12
1,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2023,1.179359e+12
2,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2022,1.228968e+12
3,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2021,1.114145e+12
4,Africa Eastern and Southern,AFE,GDP (current US$),NY.GDP.MKTP.CD,2020,9.386076e+11


In [11]:
years = [str(y) for y in range(2019,2025)]
years

['2019', '2020', '2021', '2022', '2023', '2024']

In [13]:
df_filtered = df[df['year'].isin(years)]
df_filtered['year'].unique()

<StringArray>
['2024', '2023', '2022', '2021', '2020', '2019']
Length: 6, dtype: str

In [14]:
countries_to_keep = ['Afghanistan', 'Bangladesh', 'Bhutan', 'India', 'Maldives', 'Nepal', 'Pakistan', 'Sri Lanka'] 
df_filtered = df_filtered[df_filtered['country_name'].isin(countries_to_keep)]
df_filtered['country_name'].unique()

<StringArray>
['Afghanistan',  'Bangladesh',      'Bhutan',       'India',    'Maldives',
       'Nepal',    'Pakistan',   'Sri Lanka']
Length: 8, dtype: str

In [15]:
df['year'] = df['year'].astype(int)

In [16]:
df_filtered = df_filtered.dropna(subset= ['value'])
df_filtered.shape

(281, 6)

In [22]:
df_final = df_filtered.sort_values(
    by=['country_code', 'indicator_code', 'year'],
    ascending=[True, True, True]
).reset_index(drop=True)

df_final

,country_name,country_code,indicator_name,indicator_code,year,value
0,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2019,2.302373e+00
1,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2020,5.601888e+00
2,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2021,5.133203e+00
3,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2022,1.371210e+01
4,Afghanistan,AFG,"Inflation, consumer prices (annual %)",FP.CPI.TOTL.ZG,2023,-4.644709e+00
...,...,...,...,...,...,...
276,Pakistan,PAK,"Population, total",SP.POP.TOTL,2020,2.350017e+08
277,Pakistan,PAK,"Population, total",SP.POP.TOTL,2021,2.394778e+08
278,Pakistan,PAK,"Population, total",SP.POP.TOTL,2022,2.437007e+08
279,Pakistan,PAK,"Population, total",SP.POP.TOTL,2023,2.475045e+08


In [23]:
df_final = df_final.drop_duplicates()

In [24]:
df_final.to_csv('../data/processed/south_asia_economic_metric_2018_to_2024.csv')
print('Clean WDI South Asia')

Clean WDI South Asia
